In [1]:
%%capture
# Running the previous notebooks
%run TSP.ipynb
%run TSPextended.ipynb

#Importing their classes
tsp = TSP()
tsp_ext = TSPext()

Importing the relevant packages.
We require the following:

In [2]:
import math
import pandas as pd
import numpy as np
from numpy import random as npr
import time
import copy

The function $RandomRouteGenerator$ takes a distance matrix as input, generates a set of indicies using the size of the matrix, and then uses the indices to generate a random route in the network.

In [3]:
def RandomRouteGenerator(distance_dataframe):
    
    #Determining the number of cities in our dataset
    no_of_cities = len(distance_dataframe.index)

    #Creating the city indicies 
    city_indexes = range(1, no_of_cities+1)

    #Initializing the random route as an numpy array
    random_route = np.array([0] * no_of_cities)

    #Creating a random generator
    rng = npr.default_rng()
    
    #Choosing a random city and adding it to random_route
    choose = rng.choice(city_indexes)
    random_route[0] = choose

    for i in range(no_of_cities-1):
    
        #Removing chosen city from city_indexes
        city_indexes = [node for node in city_indexes if node != choose]

        #Choosing a new random city from city_indexes and adding it to random route
        choose = rng.choice(city_indexes)
        random_route[i+1] = choose
        
    random_route = np.array(random_route)
    
    return(random_route)

The function $RestrictedRandomRouteGenerator$ takes a distance matrix as input, generates a set of indicies using the size of the matrix, and then uses the indices to generate a random route in the network, with the condition that the route starts at the depot node.

In [4]:
def RestrictedRandomRouteGenerator(distance_dataframe, depot):
    
    #Determining the number of cities in our dataset
    no_of_cities = len(distance_dataframe.index)

    #Creating the city indicies whilst removing the depot node
    city_indexes = list(range(1, depot)) + list(range(depot+1, no_of_cities+1))

    #Initializing the random route as an numpy array
    random_route = np.array([0] * no_of_cities)
    
    #Creating a random generator
    rng = npr.default_rng()

    #Choosing a random city and adding it to random_route
    choose = rng.choice(city_indexes)
    random_route[1] = choose

    for i in range(2, no_of_cities):
    
        #Removing chosen city from city_indexes
        city_indexes = [node for node in city_indexes if node != choose]

        #Choosing a new random city and adding it to random route
        choose = rng.choice(city_indexes)
        random_route[i] = choose

    #Adding depot to the beginning of random route  
    random_route[0] = depot
    
    random_route = np.array(random_route)

    return(random_route)

The function $TimeLimRandomSearch$ is a random search algorithm with a time-based termination condition. It uses $RandomRouteGenerator$ to create random tours until time $t$ whilst storing the optimal tour.

In [5]:
def TimeLimRandomSearch(distanceMatrix, total_time):
    
    #Generating a random route
    initial_route = RandomRouteGenerator(distanceMatrix)
    
    #Determining the cost of the route
    initial_cost = tsp.getCostofRoute(initial_route, distanceMatrix)
    
    start = time.time()
    
    #Generate solutions until total_time has passed while storing the optimal solution.
    while ((time.time() - start) < total_time):
        route = RandomRouteGenerator(distanceMatrix)
        cost = tsp.getCostofRoute(initial_route, distanceMatrix)
        if cost < initial_cost:
            initial_cost = cost
            initial_route = route
    
    return((initial_route, initial_cost))

The function $twoOptSwap$ generates the 2-opt neighbourhood for a given route, i.e. the set of all routes reachable by a 2-opt swap.

In [6]:
def twoOptSwap(route):
    #math.comb(len(route), 2) returns nC2, where n is the length of the route. 
    #This is the total number of subsets of size 2 we can choose from a set with
    #integers 1 to n. Since we compare each pair of index twice ((i,j) and (j,i))
    #we need the number of rows in our neighbourhood to be twice this value.
    N = 2* math.comb(len(route), 2)
    
    #Define an empty array to contain all neighbours
    route_collection = np.zeros((N, len(route)))
    
    new_route = copy.deepcopy(route)
    
    #Indicates position of row in matrix
    counter = 0
    
    #Iterating in order to compare element j to every other element k
    for j in range(len(new_route)):
        m = new_route[j]
         
        for k in range(len(new_route)):
            
            #Ensuring the indices are not equal
            if k != j:
                
                #Performing the 2-opt swap
                n =  new_route[k]
                
                new_route[k] = m
                
                new_route[j] = n
                
                #Adding the route to the neighbourhood
                route_collection[counter] = np.array(new_route)
    
                counter = counter + 1
                
            new_route = copy.deepcopy(route)

    return(route_collection)         

The function $bestNeighbourStep$ evaluates all the routes in a 2-opt neighbourhood of a selected route (using the $twoOptSwap$ function), and then returns the optimal neighbour alongside its associated cost.

In [7]:
def bestNeighbourStep(route, distance_dataframe):
    
    #Generating the 2-opt neighbourhood of the solution
    neighbourhood = twoOptSwap(route)
    
    #Determing the most optimal neighbour
    next_step = tsp.bestRoute(neighbourhood, distance_dataframe)
    
    return(next_step)

The function $twoOptsearchAlgorithm$ repeatedly tries to make the best move using the local 2-opt neighbourhood for a random route route, and repeats this process until it produces a 2-opt neighbourhood which has only more expensive routes, (i.e. it reaches an optimum value).

In [8]:
def twoOptsearchAlgorithm(distance_dataframe):
    
    #Generate a random route from the distance matrix
    randomRoute = RandomRouteGenerator(distance_dataframe)
    
    initial_cost = tsp.getCostofRoute(randomRoute, distance_dataframe)
    
    #Determine the optimal neighbour in the 2opt neighbourhood of the random route as well as its associated cost
    next_step = bestNeighbourStep(randomRoute, distance_dataframe)
    updated_route = next_step[1]
    updated_cost = next_step[0]

    while(updated_cost < initial_cost):
        
        initial_cost = updated_cost
        
        ##Determine the optimal neighbour in the 2opt neighbourhood of updated_route
        next_step = bestNeighbourStep(updated_route, distance_dataframe)
        
        updated_route = next_step[1]
        updated_cost = next_step[0]
    
    return(next_step)

The function $randomtwoOptSearchAlgorithm$ applies $twoOptsearchAlgorithm$ until it obtains a final value, after which it repeats the search from a newly generated random tour whilst keeping track of the global optimum value it has obtained. It continoues this process until a specified time limit has elapsed, and then returns the obtained global optimum value.

In [9]:
def randomtwoOptSearchAlgorithm(distance_dataframe, total_time):
    start = time.time()
    
    #Obtains first local optimum
    initial_cost = twoOptsearchAlgorithm(distance_dataframe)
    
    #Improves upon local optimum by repeating the process until total_time has elapsed
    while ((time.time() - start) < total_time):
        updated_cost = twoOptsearchAlgorithm(distance_dataframe)
        if updated_cost < initial_cost:
            initial_cost = updated_cost
        else:
            initial_cost = initial_cost
    
    return(initial_cost)

We can now define the class $LocalSearch$.

In [10]:
class LocalSearch:
    def __init__(self):
        pass
    
    def createRandomRoute(self, distance_dataframe, restricted = False, depot = 1):
        if restricted is False:
            random_route = RandomRouteGenerator(distance_dataframe)
        else:
            random_route = RestrictedRandomRouteGenerator(distance_dataframe, depot)
            return(random_route)
    
    def BestNeighbourtwoOpt(self, route, distance_dataframe):
        neighbourhood = twoOptSwap(route)
        best_step = bestNeighbourStep(route, distance_dataframe)
        return(neighbourhood, best_step)
    
    def Search(self, distance_dataframe, method = "Random", total_time=1):
        if method == "Random":
            solution = TimeLimRandomSearch(distance_dataframe, total_time)
        elif method == "2opt":
            solution = twoOptsearchAlgorithm(distance_dataframe)
        elif method == "Random_2opt":
            solution = randomtwoOptSearchAlgorithm(distance_dataframe, total_time)
        return(solution)    

### Testing

Let us test the functions defined in this file.

In [11]:
#Importing the distance dataframe for testing
columns = [i for i in range(1,26)]
distance_dataframe_test = pd.read_csv('distance_dataframes/dist_dataframe_normal_1_25_wo:9.csv',names = columns)
distance_dataframe_test = distance_dataframe_test.iloc[1:]

In [12]:
#Creating a route
RandomRouteGenerator(distance_dataframe_test)

array([22,  4, 23,  2, 25, 18,  3,  9,  7, 21,  6, 20, 16, 11, 13,  1, 12,
       19,  5, 17, 10,  8, 15, 24, 14])

In [15]:
#Creating a restricted route
tmp = RestrictedRandomRouteGenerator(distance_dataframe_test, 1)
tmp

array([ 1, 20,  6, 17, 24,  4, 25,  3, 15, 22,  2,  9,  7, 23, 13, 11, 14,
       21, 19,  5, 16, 18,  8, 12, 10])

In [14]:
#Testing random search
TimeLimRandomSearch(distance_dataframe_test, 30)

(array([ 1, 17, 20, 24, 11, 13,  6, 16, 14,  8, 18,  3,  9, 10, 23, 21,  2,
         7,  5, 25, 22, 15, 19,  4, 12]),
 11.57)

In [18]:
#Testing twoOptSwap
tmp2 = twoOptSwap(tmp)
tmp2

array([[20.,  1.,  6., ...,  8., 12., 10.],
       [ 6., 20.,  1., ...,  8., 12., 10.],
       [17., 20.,  6., ...,  8., 12., 10.],
       ...,
       [ 1., 20.,  6., ...,  8., 12., 18.],
       [ 1., 20.,  6., ..., 10., 12.,  8.],
       [ 1., 20.,  6., ...,  8., 10., 12.]])

In [20]:
len(tmp2)

600

In [21]:
bestNeighbourStep(tmp, distance_dataframe_test)

(10.24,
 array([ 1., 20.,  6., 21., 24.,  4., 25.,  3., 15., 22.,  2.,  9.,  7.,
        23., 13., 11., 14., 17., 19.,  5., 16., 18.,  8., 12., 10.]))

In [22]:
randomtwoOptSearchAlgorithm(distance_dataframe_test, 30)

(4.6499999999999995,
 array([25.,  5.,  7., 14.,  4., 10.,  3., 17., 19., 24.,  1., 16., 15.,
        13.,  2., 23., 18., 22., 11., 12.,  8.,  6., 21.,  9., 20.]))